## **Setting up Development Environment**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1


In [ ]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

     |████████████████████████████████| 4.4 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 44.9 MB/s 
     |████████████████████████████████| 596 kB 58.2 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 1.2 MB 4.4 MB/s 


## **Importing Libraries**

In [ ]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from imblearn.over_sampling import SMOTE
from collections import Counter

from transformers import BertModel, BertTokenizer
from transformers import AutoTokenizer, AutoModel

import pickle

## **Setting up Stemmer**

In [ ]:
! rm -r tamil-stemmer-build
! rm input.txt
! rm output.txt

! mkdir tamil-stemmer-build
% cd tamil-stemmer-build
! git clone https://github.com/rdamodharan/tamil-stemmer.git
! tar -zxf tamil-stemmer/snowball-with-tamil.tgz
% cd snowball-with-tamil
! make

% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1/tamil-stemmer-build
Cloning into 'tamil-stemmer'...
remote: Enumerating objects: 88, done.
remote: Total 88 (delta 0), reused 0 (delta 0), pack-reused 88
Unpacking objects: 100% (88/88), done.
/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1/tamil-stemmer-build/snowball-with-tamil
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/space.o compiler/space.c
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/tokeniser.o compiler/tokeniser.c
In file included from compiler/tokeniser.c:6:0:
compiler/tokeniser.c: In function ‘read_token’:
compiler/header.h:8:21: warning: this statement may fall through [-Wimplicit-fallthrough=]
 #define unless(C) if(!(C))
                     ^
compiler/tokeniser.c:390:16: note: in expansion of macro ‘unless’
                unless (t->next == 0) {
               

## **Loading Data**

In [ ]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [ ]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [ ]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam1,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [ ]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [ ]:
dataframe.shape

(5880, 3)

In [ ]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [ ]:
dataframe.shape

(5877, 3)

In [ ]:
text = dataframe['text']

In [ ]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [ ]:
label = dataframe['category']

In [ ]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [ ]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [ ]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

In [ ]:
text = text.tolist()

! rm input.txt
! rm output.txt

for i in text:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

rm: cannot remove 'input.txt': No such file or directory
rm: cannot remove 'output.txt': No such file or directory


In [ ]:
! ./tamil-stemmer-build/snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [ ]:
text_file = open("output.txt", "r")

text = []

for line in text_file:
    text.append(line.strip())

In [ ]:
# text = np.array(text)
# text

## **Feature Extraction**

In [ ]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModel.from_pretrained("ai4bharat/indic-bert")

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'sop_classifier.classifier.bias', 'predictions.dense.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [ ]:
print(tokenized_input)

{'input_ids': tensor([[     2,   4436, 139013, 105466,   1385,  30231,  40291,  39555,  82156,
         150078, 194115,   5343, 104252,  82076,      3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
print(sample_output)

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.0101, -0.0098, -0.0160,  ..., -0.0302, -0.0208, -0.0186],
         [ 0.2039,  0.1569, -0.2342,  ..., -0.1102,  0.2575,  0.0765],
         [ 0.6726, -0.2458, -0.0159,  ...,  0.3238,  0.2636, -0.0796],
         ...,
         [ 0.9273, -0.2617, -0.1268,  ...,  0.2491,  0.1701,  0.0846],
         [ 0.5355, -0.0578, -0.0928,  ...,  0.2653,  0.0409, -0.5193],
         [-0.0101, -0.0098, -0.0160,  ..., -0.0302, -0.0208, -0.0186]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-5.2381e-02,  5.5714e-02,  1.6412e-02,  6.1749e-04,  5.0623e-02,
          8.1442e-02,  4.2414e-02, -7.5104e-03, -1.2329e-02,  6.2801e-02,
         -1.0946e-02,  4.8848e-02,  1.6424e-02, -2.0096e-02,  4.9712e-02,
         -1.5399e-02, -4.3242e-02,  2.2941e-03, -1.2076e-01, -8.4439e-02,
          7.8569e-02,  7.2158e-02, -3.3552e-02, -9.9965e-02,  9.4613e-04,
         -3.3801e-02, -5.7639e-02, -6.0294e-03,  2.2355e-02,  7.2021e-03,
          2

In [ ]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [ ]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [ ]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./TrainData/output1.pickle done
./TrainData/output2.pickle done
./TrainData/output3.pickle done
./TrainData/output4.pickle done
./TrainData/output5.pickle done
./TrainData/output6.pickle done
./TrainData/output7.pickle done
./TrainData/output8.pickle done
./TrainData/output9.pickle done
./TrainData/output10.pickle done
./TrainData/output11.pickle done
./TrainData/output12.pickle done
./TrainData/output13.pickle done
./TrainData/output14.pickle done
./TrainData/output15.pickle done
./TrainData/output16.pickle done
./TrainData/output17.pickle done
./TrainData/output18.pickle done
./TrainData/output19.pickle done
./TrainData/output20.pickle done
./TrainData/output21.pickle done
./TrainData/output22.pickle done
./TrainData/output23.pickle done
./TrainData/output24.pickle done
./TrainData/output25.pickle done
./TrainData/output26.pickle done
./TrainData/output27.pickle done
./TrainData/output28.pickle done
./TrainData/output29.pickle done
./TrainData/output30.pickle done


In [ ]:
# Loading Embedded Input Data from Disk

output = []

for i in range(30):
    file_name = "./TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./TrainData/output1.pickle done
./TrainData/output2.pickle done
./TrainData/output3.pickle done
./TrainData/output4.pickle done
./TrainData/output5.pickle done
./TrainData/output6.pickle done
./TrainData/output7.pickle done
./TrainData/output8.pickle done
./TrainData/output9.pickle done
./TrainData/output10.pickle done
./TrainData/output11.pickle done
./TrainData/output12.pickle done
./TrainData/output13.pickle done
./TrainData/output14.pickle done
./TrainData/output15.pickle done
./TrainData/output16.pickle done
./TrainData/output17.pickle done
./TrainData/output18.pickle done
./TrainData/output19.pickle done
./TrainData/output20.pickle done
./TrainData/output21.pickle done
./TrainData/output22.pickle done
./TrainData/output23.pickle done
./TrainData/output24.pickle done
./TrainData/output25.pickle done
./TrainData/output26.pickle done
./TrainData/output27.pickle done
./TrainData/output28.pickle done
./TrainData/output29.pickle done
./TrainData/output30.pickle done


In [ ]:
X = np.array(X)
X.shape

(5877, 1, 768)

In [ ]:
X = X.reshape(5877, 768)
X.shape

(5877, 768)

In [ ]:
y = label
y.shape

(5877,)

## **Train Test Split**

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

(4701, 768)

In [ ]:
X_val.shape

(1176, 768)

## **SMOTE Oversampling**

In [ ]:
counter = Counter(y_train)
print(counter)

Counter({0: 3774, 1: 927})


In [ ]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [ ]:
counter = Counter(y_train)
print(counter)

Counter({0: 3774, 1: 3774})


## **Logistic Regression**

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.87      0.68      0.76       950
           1       0.30      0.57      0.39       226

    accuracy                           0.66      1176
   macro avg       0.58      0.62      0.58      1176
weighted avg       0.76      0.66      0.69      1176



## **Support Vector Machine**

In [ ]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.86      0.70      0.77       950
           1       0.30      0.53      0.38       226

    accuracy                           0.67      1176
   macro avg       0.58      0.61      0.58      1176
weighted avg       0.75      0.67      0.70      1176



## **Naive Bayes**

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.86      0.66      0.74       950
           1       0.28      0.55      0.37       226

    accuracy                           0.64      1176
   macro avg       0.57      0.60      0.56      1176
weighted avg       0.75      0.64      0.67      1176



## **Stochastic Gradient Descent**

In [ ]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.85      0.86      0.85       950
           1       0.38      0.36      0.37       226

    accuracy                           0.76      1176
   macro avg       0.61      0.61      0.61      1176
weighted avg       0.76      0.76      0.76      1176



## **K Nearest Neighbours**

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.91      0.49      0.64       950
           1       0.27      0.79      0.40       226

    accuracy                           0.55      1176
   macro avg       0.59      0.64      0.52      1176
weighted avg       0.79      0.55      0.60      1176



## **Decision tree**

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.84      0.74      0.78       950
           1       0.27      0.41      0.33       226

    accuracy                           0.67      1176
   macro avg       0.56      0.57      0.56      1176
weighted avg       0.73      0.67      0.70      1176



## **Random Forest**

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.84      0.89      0.86       950
           1       0.39      0.30      0.33       226

    accuracy                           0.77      1176
   macro avg       0.61      0.59      0.60      1176
weighted avg       0.75      0.77      0.76      1176



## **Majority Voting**

In [ ]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Keeping only DT, KNN & NB in predictions due to poor performance
    predictions = [dt_pred[i], knn_pred[i], rf_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))    

              precision    recall  f1-score   support

           0       0.86      0.75      0.80       950
           1       0.32      0.49      0.39       226

    accuracy                           0.70      1176
   macro avg       0.59      0.62      0.60      1176
weighted avg       0.76      0.70      0.72      1176



## **Loading Testing Data**

In [ ]:
dataframe_test = pd.read_csv(
    "Dataset/tam_offesive_withoutlabels_test.tsv", 
    sep='\t'
)
dataframe_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [ ]:
dataframe_test.describe()

,id,text
count,654,654
unique,654,654
top,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
freq,1,1


In [ ]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [ ]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
text_test = text_test.tolist()

! rm input.txt
! rm output.txt

for i in text_test:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

In [ ]:
! ./tamil-stemmer-build/snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [ ]:
text_file = open("output.txt", "r")

text_test = []

for line in text_file:
    text_test.append(line.strip())

text_test = np.array(text_test)

## **Test Feature Extraction**

In [ ]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModel.from_pretrained("ai4bharat/indic-bert")

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'sop_classifier.classifier.bias', 'predictions.dense.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Sample Output

tokenized_input = tokenizer(
        text_test[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [ ]:
print(tokenized_input)

{'input_ids': tensor([[     2,  20264,   3048,  92072,  52721,   1385,   1385,  49940,   1657,
          32885,  13143, 124420,   3941,  10885,  20264,   3048,  34875,  15528,
              8,  41858,   6119,  14359,   3534,      3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
print(sample_output)

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.0220, -0.0103, -0.0184,  ..., -0.0277, -0.0151, -0.0177],
         [ 0.1349,  0.2533, -0.5680,  ..., -0.0561, -0.0241,  0.1819],
         [ 0.3869,  0.3930, -0.0643,  ...,  0.1082,  0.0555,  0.2692],
         ...,
         [ 0.5151,  0.1412, -0.3426,  ..., -0.0786,  0.3912,  0.5449],
         [ 0.6777, -0.0342,  0.0573,  ..., -0.0871,  0.1859,  0.0947],
         [-0.0220, -0.0103, -0.0184,  ..., -0.0277, -0.0151, -0.0177]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-5.1660e-02,  5.3390e-02,  1.7966e-02, -4.3926e-03,  4.7348e-02,
          8.4199e-02,  3.7961e-02, -7.8548e-03, -1.4522e-02,  5.7318e-02,
         -9.9887e-03,  4.9222e-02,  1.3639e-02, -2.0439e-02,  5.5387e-02,
         -1.7512e-02, -4.5300e-02,  4.5599e-03, -1.1802e-01, -8.8100e-02,
          8.2153e-02,  7.8851e-02, -3.3273e-02, -9.7838e-02,  9.7679e-04,
         -3.3552e-02, -5.4551e-02, -7.7866e-03,  2.3278e-02,  1.1118e-02,
          2

In [ ]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [ ]:
# Tokenizing Test Input Data

input = []

for i in text_test:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [ ]:
# Embedding the Test Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Test Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./TestData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./TestData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./TestData/output1.pickle done
./TestData/output2.pickle done
./TestData/output3.pickle done
./TestData/output4.pickle done


In [ ]:
# Loading Embedded Input Test Data from Disk

output = []

for i in range(4):
    file_name = "./TestData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X_test = output
output = []

./TestData/output1.pickle done
./TestData/output2.pickle done
./TestData/output3.pickle done
./TestData/output4.pickle done


In [ ]:
X_test = np.array(X_test)
X_test.shape

(654, 1, 768)

In [ ]:
X_test = X_test.reshape(654, 768)
X_test.shape

(654, 768)

## **Getting Predictions**

In [ ]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Support Vector Machine
svm_test_pred = svm.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

# Stochastic Gradient Descent
sgd_test_pred = sgd.predict(X_test)

# K Nearest Neighbours
knn_test_pred = knn.predict(X_test)

# Decision Tree
dt_test_pred = dt.predict(X_test)

# Random Forest
rf_test_pred = rf.predict(X_test)

In [ ]:
# Majority Voting
mv_test_pred = []

for i in range(len(lr_test_pred)):
    one = 0
    zero = 0
    predictions = [rf_test_pred[i], knn_test_pred[i], dt_test_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_test_pred.append(1)
    else: mv_test_pred.append(0)

mv_test_pred = np.array(mv_test_pred)

## **Evaluating the Predictions**

In [ ]:
dataframe_eval = pd.read_csv(
    "Dataset/tamil_offensive_test task 1 with labels.tsv", 
    sep='\t'
)

In [ ]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [ ]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred))

Logistic Regression
              precision    recall  f1-score   support

           0       0.91      0.71      0.80       536
           1       0.35      0.69      0.46       118

    accuracy                           0.71       654
   macro avg       0.63      0.70      0.63       654
weighted avg       0.81      0.71      0.74       654



In [ ]:
print('Support Vector Machine')
print(classification_report(y_test, svm_test_pred))

Support Vector Machine
              precision    recall  f1-score   support

           0       0.91      0.72      0.80       536
           1       0.34      0.66      0.45       118

    accuracy                           0.71       654
   macro avg       0.62      0.69      0.63       654
weighted avg       0.80      0.71      0.74       654



In [ ]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

Naive Bayes
              precision    recall  f1-score   support

           0       0.90      0.68      0.77       536
           1       0.31      0.66      0.42       118

    accuracy                           0.67       654
   macro avg       0.61      0.67      0.60       654
weighted avg       0.79      0.67      0.71       654



In [ ]:
print('Stochastic Gradient Descent')
print(classification_report(y_test, sgd_test_pred))

Stochastic Gradient Descent
              precision    recall  f1-score   support

           0       0.87      0.86      0.86       536
           1       0.39      0.42      0.40       118

    accuracy                           0.78       654
   macro avg       0.63      0.64      0.63       654
weighted avg       0.78      0.78      0.78       654



In [ ]:
print('K Nearest Neighbours')
print(classification_report(y_test, knn_test_pred))

K Nearest Neighbours
              precision    recall  f1-score   support

           0       0.93      0.51      0.66       536
           1       0.27      0.82      0.41       118

    accuracy                           0.56       654
   macro avg       0.60      0.66      0.53       654
weighted avg       0.81      0.56      0.61       654



In [ ]:
print('Decision Tree')
print(classification_report(y_test, dt_test_pred))

Decision Tree
              precision    recall  f1-score   support

           0       0.86      0.74      0.80       536
           1       0.28      0.46      0.35       118

    accuracy                           0.69       654
   macro avg       0.57      0.60      0.57       654
weighted avg       0.76      0.69      0.72       654



In [ ]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred))

Random Forest
              precision    recall  f1-score   support

           0       0.86      0.89      0.88       536
           1       0.42      0.36      0.38       118

    accuracy                           0.79       654
   macro avg       0.64      0.62      0.63       654
weighted avg       0.78      0.79      0.79       654



In [ ]:
print('Majority Voting')
print(classification_report(y_test, mv_test_pred))

Majority Voting
              precision    recall  f1-score   support

           0       0.88      0.77      0.82       536
           1       0.34      0.54      0.42       118

    accuracy                           0.73       654
   macro avg       0.61      0.66      0.62       654
weighted avg       0.79      0.73      0.75       654

